In [30]:
import pandas as pd
import re

In [31]:
# pd.set_option('display.max_colwidth', None)

In [32]:
raw_df = pd.read_csv(r'C:\Users\niyai\git\british-analysis\British_Airway_Review.csv')

In [33]:
df = raw_df

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   reviews            2500 non-null   object
 1   date               2500 non-null   object
 2   country            2500 non-null   object
 3   seat_type          2500 non-null   object
 4   recommended        2500 non-null   object
 5   stars              2500 non-null   int64 
 6   route              2500 non-null   object
 7   type_of_traveller  2500 non-null   object
dtypes: int64(1), object(7)
memory usage: 156.4+ KB


In [35]:
df.head(1)

,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller
0,✅ Trip Verified | I had the most fantastic BA Flight today. The cabin crew in my seat zone 6D were of the best I have experienced. Although the 777 had the old style rear/front seating - it was comfortable and felt open and spacious. BA have done great things with the menu - it was nice not to have to eat breakfast food on the pre-arrival meal into HKG at 1730PM local time having left London at 2200 the night before. The dinner offering was equally ample and delicious. The inflight entertainment had a great collection of TV and current films and the inflight map was first rate. Previously not a huge BA fan - but after this 12 hour flight I could only recommend it without reserve. I flew the new BA suite in J class on a recent Delhi to London flight and enjoyed that product (although I felt it slightly more cramped ) but I think BA have stepped up with really comfortable bedding and food and very friendly crew who engaged throughout the flight in the best way. Couldn't recommend BA more on this flight today,1st August 2023,Hong Kong,Business Class,yes,5,Heathrow to Las Vegas,Family Leisure


In [36]:
df.seat_type.unique()

array(['Business Class', 'Economy Class', 'Premium Economy',
       'First Class'], dtype=object)

In [37]:
df.type_of_traveller.unique()

array(['Family Leisure', 'Solo Leisure', 'Couple Leisure', 'Business'],
      dtype=object)

In [38]:
# Function to remove ordinal indicators from datetime strings
# Remove ordinal indicators from the 'date' column
def remove_ordinal_indicator(date_str):
    return re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)

df['date'] = df['date'].apply(remove_ordinal_indicator)

In [39]:
# Convert the 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'], format='%d %B %Y')

In [40]:
df['ym'] = df['date'].astype(str)
df['ym'] = df['ym'].str[:7]

In [41]:
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [42]:
df['from'] = df['route'].str.split(' to ', expand = True)[0]
df['to'] = df['route'].str.split(' to ', expand = True)[1]

In [43]:
df['verified'] = df.reviews.str.startswith('✅ Trip Verified', '✅ Verified Review')

In [44]:
prefix = ( # verified
        '✅ Trip Verified'
          , '✅ Verified Review'
          # unverified
          , 'Not Verified'
          , '❎ Not Verified'
          , '❎ Unverified')

In [45]:
# Custom function to check if a string starts with a prefix
def contains_prefix(row):
    return any(row['reviews'].startswith(prefix) for prefix in prefix)

# Apply the custom function to each row and store the result in a new column
df['contains_prefix'] = df.apply(contains_prefix, axis=1)


In [46]:
bool_to_yes_no = {True: 'Yes', False: 'No'}

# Use mapping dictionary to replace values in the type_traveller column
df['contains_prefix'] = df['contains_prefix'].map(bool_to_yes_no)
df['verified'] = df['verified'].map(bool_to_yes_no)

In [47]:
# Character after which you want to split the string
character ='|' 

# Custom function to split the string and select values after the character
def extract_value(value, character):
    if character in value:
        return value.split(character, 1)[-1]
    else:
        return value

In [48]:
df['cleaned_reviews'] = df['reviews'].apply(lambda x: extract_value(x, character))

In [49]:
newdf = df.drop(columns = ['reviews', 'route'])

In [50]:
# newdf

In [51]:
newdf.to_csv('British_Airway_Review_cleaned.csv', index = False)

In [52]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               2500 non-null   datetime64[ns]
 1   country            2500 non-null   object        
 2   seat_type          2500 non-null   object        
 3   recommended        2500 non-null   object        
 4   stars              2500 non-null   int64         
 5   type_of_traveller  2500 non-null   object        
 6   ym                 2500 non-null   object        
 7   day                2500 non-null   int32         
 8   month              2500 non-null   int32         
 9   year               2500 non-null   int32         
 10  from               2500 non-null   object        
 11  to                 2500 non-null   object        
 12  verified           2500 non-null   object        
 13  contains_prefix    2500 non-null   object        
 14  cleaned_